In [26]:
import os
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from PIL import Image
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from fuzzywuzzy import process, fuzz

This Script assumes that a .pdf file was already segmented to a folder "base_path". It also assumes there are the labels to the segments in an excel sheet. The segments are colored RGB images, to allow ColorChecks and QuotationmarkChecks

In [2]:
base_path='C:\\Uni\\1M. Semester\\DocDig\\gen_img\\scan_1972_CdB_3_20231125160810.pdf'

Read In the Excel Sheet, discard the first two and the last segment of each page (-0.png, -1.png, -last.png)

In [3]:
label_path = '125160810Gelabelt.xlsx'
excel_data = pd.ExcelFile(os.path.join(base_path, label_path))

# Initialize a list to store labels for each sheet
label_list = []

# Iterate through each sheet in the Excel file
for sheet_name in excel_data.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(os.path.join(base_path, label_path), header=None, sheet_name=sheet_name, dtype=str)
    
    # Extract the first 50 lines from the second column (column 'B')
    labels = df.iloc[:50, 1]  # Get the first 50 rows of column B
    
    # Replace NaN values (empty cells) with empty strings
    labels = labels.fillna('')
    
    # Append the labels as a list for this sheet, preserving all values
    label_list.append(labels.values)

print(label_list)

[array(['Mehlschwalbe', '"', '"', '"', '"', '"', '"', '"', '"', '"', '"',
       'Tannenmeise', '"', '"', 'Erlenzeisig', '"', '"', '"', 'Blaumeise',
       '"', 'Mehlschwalbe', '"', 'Tannenmeise', '"', '"', '"', '"', '"',
       '"', '"', '"', 'Blaumeise', 'Tannenmeise', 'Blaumeise', '"',
       'Tannenmeise', '"', '"', 'Schafstelze', 'Mehlschwalbe',
       'Tannenmeise', '"', '"', '"', '"', '"', '"', '"', '"', '"'],
      dtype=object), array(['Tannenmeise', '"', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', 'Mehlschwalbe',
       'Tannenmeise', '"', 'Blaumeise', 'Schafstelze', 'Blaumeise',
       'Tannenmeise', '"', 'Blaumeise', 'Tannenmeise', '"', '"', '"', '"',
       '"', 'Blaumeise', 'Tannenmeise', '"', '"', '"', '"', '"', '"', '"',
       '"', '"'], dtype=object), array(['Tannenmeise', '', '', '', '', 'Mehlschwalbe', 'Blaumeise', '"',
       'Tannenmeise', '"', 'Blaumeise', 'Tannenmeise', '"', 'Blaumeise',
       '"', '"', 'Tannen

In [4]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

C:\Programmieren\Python\DocDigTransformer\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Luca Scagliola\.cache\huggingface\hub\models--microsoft--trocr-large-handwritten. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# extract classes
import json

# Load the JSON file
with open("./class_indices_extended.json", "r", encoding="utf-8") as file:
    class_indices = json.load(file)

# Extract the class names (ignoring the index)
class_names = list(class_indices.keys())

# Display the class names
print("Class Names:", class_names)

Class Names: ['"', 'Alpenmeise', 'Baumpieper', 'Birkenzeisig', 'Blaumeise', 'Braunkelchen', 'Distelfink', 'Erlenzeisig', 'Fitis', 'Gartenrotschwanz', 'Gartenrötel', 'Gelbspöter', 'Girlitz', 'Grauschnäpper', 'Hausrotschwanz', 'Hausrötel', 'Hänfling', 'Heckenbraunelle', 'Klappergrasmücke', 'Rotkelchen', 'Tannenmeise', 'Trauerschnäpper', 'Wintergoldhähnchen', 'Zaungrasmücke', 'Zilpzalp', 'Zitroneler', 'Zitronenfink', 'Zitronenzeisig', 'Mehlschwalbe', 'Rauchschwalbe', 'Schafstelze', 'Teichrohrsänger', 'Uferschwalbe', 'Gobemouche noir', 'Linotte mélodieuse', 'Mésange bleu', 'Mésange noir', 'Pipit des arbres', 'Pouillot véloce', 'Rougegorge', 'Rougequeue à front blanc', 'Traquet tarier', 'Bergeronnette grise', 'Accenteur mouchet', 'Tarin', 'Serin']


In [56]:
# Replace numbers with letters
# Define the mappings from numbers to similar-looking letters
number_to_letter_map = {
    '0': 'O',  # zero to uppercase 'O'
    '1': 'I',  # one to uppercase 'I'
    '2': 'Z',  # two to uppercase 'Z'
    '3': 'E',  # three to uppercase 'E'
    '4': 'A',  # four to uppercase 'A'
    '5': 'S',  # five to uppercase 'S'
    '6': 'G',  # six to uppercase 'G'
    '7': 'T',  # seven to uppercase 'T'
    '8': 'B',  # eight to uppercase 'B'
    '9': 'P',  # nine to uppercase 'P'
}

# Replace numbers with similar-looking letters
def replace_numbers_with_letters(text):
    # Substitute each number with its corresponding letter
    for num, letter in number_to_letter_map.items():
        text = text.replace(num, letter)
    return text

# Reward for same initial letter
def custom_fuzzy_match(query, bird_names, prefix_weight=1.25):
    best_match = None
    highest_score = 0

    for bird_name in bird_names:
        # Berechnen Sie die Fuzzy-Übereinstimmung
        score = fuzz.ratio(query, bird_name)

        # Fügen Sie eine Gewichtung hinzu, wenn der Anfangsbuchstabe übereinstimmt
        if bird_name.lower().startswith(query[0].lower()):
            score *= prefix_weight

        # Aktualisieren Sie das beste Ergebnis
        if score > highest_score:
            highest_score = score
            best_match = bird_name

    return best_match, highest_score

# Function to determine the predominant color in an image
def check_text_color(image):
    
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Get the height, width, and channels of the image
    height, width, _ = image.shape

    # Define the HSV range for blue
    lower_blue = np.array([100, 50, 50])  # Lower bound of the blue range
    upper_blue = np.array([140, 255, 255])
    
    # Create mask
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Count the number of pixels in each mask
    blue_count = np.sum(mask_blue > 0)

    # Determine if text is predominantly black or blue
    total_pixels = height * width
    blue_ratio = blue_count / total_pixels
    
    # Only Blue Pixels
    imask = mask_blue>0
    blue = np.full((height, width, _), 255, dtype=np.uint8)
    blue[imask] = image[imask]

    #plt.imshow(cv.cvtColor(blue, cv.COLOR_BGR2RGB))
    #plt.show()

    # Print the results for insight
    return {
        'blue_ratio': blue_ratio,
        'predominant_color': 'blue' if blue_ratio > 0.02 else 'black'
    }

# Function to replace all black pixels with white in an image
def remove_black_pixels(image): # detect all non-black pixels
    
    # Get the height, width, and channels of the image
    height, width, _ = image.shape

    # Define the HSV range for all colors except black and near-black
    lower_black = np.array([0, 50, 50])  # Lower bound to exclude black and near-black
    upper_black = np.array([179, 255, 255])  # Upper bound for all colors

    ## Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_black, upper_black)

    ## Slice
    imask = mask>0
    white = np.full((height, width, _), 255, dtype=np.uint8)
    white[imask] = image[imask]

    return white

def detectQuotationMarks(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for "nearly white"
    lower_white = np.array([0, 0, 200])  # Low saturation and high value
    upper_white = np.array([180, 50, 255])           # Wide range of hue and low saturation

    # Create a mask for pixels that are considered white or nearly white
    mask = cv2.inRange(hsv, lower_white, upper_white)

    # Invert the mask to count non-white pixels
    non_white_pixels = np.sum(mask == 0)

    # Get the total number of pixels in the image
    total_pixels = hsv.shape[0] * hsv.shape[1]

    # Calculate the relative amount of non-white pixels
    relative_non_white = non_white_pixels / total_pixels

    # Determine if the image contains text or just quotation marks based on the ratio
    if relative_non_white < 0.1157:
        return {
            'Quotationmark': True,
            'relativeNonWhite': relative_non_white
        }
    else:
        return {
            'Quotationmark': False,
            'relativeNonWhite': relative_non_white
        }

In [60]:
# Define the path to the folder containing your images
image_folder = "NOPRE-COLOR-col-1"  # Replace with the actual folder path
# Array to store the OCR outputs
ocr_outputs = []
threshold = 0

for page in range(len(label_list)): # for all pages
    for line in range(50):
        filename = str(page) + "-" + str(line+2) + ".png"
        
        label = label_list[page][line]
        
        # Construct the full file path
        file_path = os.path.join(base_path, "cells", image_folder, filename)

        # Check if the file is an image (you can add more extensions if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            try:
                print(file_path)
                # Load and process the image
                #image = Image.open(file_path).convert("RGB")
                image_original = cv2.imread(file_path)
                
                # Check colors of original image (remove black if text is written in blue)
                tmp1 = check_text_color(image_original)
                if tmp1['predominant_color'] == 'blue':
                    image = remove_black_pixels(image_original)
                else:
                    image = image_original
                    
                # Check quotationmarks (no further processing needed, if quotationmark detected)
                tmp2 = detectQuotationMarks(image_original)
                
                # The network needs a PIL image
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                image = Image.fromarray(image)  # Convert NumPy array to PIL Image
                
                if tmp2['Quotationmark'] == False:
                
                    pixel_values = processor(images=image, return_tensors="pt").pixel_values

                    # Generate text from the OCR model
                    generated_ids = model.generate(pixel_values)#, output_scores=True, return_dict_in_generate=True)
                    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

                    # Fuzzy match the recognized text to the bird species list
                    best_match, score = process.extractOne(generated_text, class_names)

                    # Post-process to filter only a-z and A-Z characters in the recognized text
                    filtered_text = re.sub(r'[^a-zA-ZäöüÄÖÜß"]', '', replace_numbers_with_letters(generated_text))

                    # Fuzzy match the processed recognized text to the bird species list
                    best_match_filtered, score_filtered = process.extractOne(filtered_text, class_names)
                
                    # Fuzzy match the processed recognized text to the bird species with initial letter bonus
                    best_match_customFuzzy, score_custom_fuzzy = custom_fuzzy_match(filtered_text, class_names)

                    # Append the result to the output array
                    ocr_outputs.append({"image": image,
                                    "generated_ids": generated_ids,
                                    "true_text": label,
                                    "detected_color": tmp1['predominant_color'],
                                    "detected_quoMark": tmp2['Quotationmark'],
                                    "recognized_text": generated_text,
                                    "best_match": best_match if score >= threshold else None,
                                    "score": score,
                                    "filtered_text": filtered_text,
                                    "best_match_filtered": best_match_filtered,
                                    "score_filtered": score_filtered,
                                    "best_match_customFuzzy": best_match_customFuzzy,
                                    "score_customFuzzy": score_custom_fuzzy})
                
                else:
                    # Append the result to the output array
                    ocr_outputs.append({"image": image,
                                    "generated_ids": "",
                                    "true_text": label,
                                    "detected_color": tmp1['predominant_color'],
                                    "detected_quoMark": tmp2['Quotationmark'],
                                    "recognized_text": "\"",
                                    "best_match": "\"",
                                    "score": 100,
                                    "filtered_text": "\"",
                                    "best_match_filtered": "\"",
                                    "score_filtered": 100,
                                    "best_match_customFuzzy": "\"",
                                    "score_customFuzzy": 100})

            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                

C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-2.png


C:\Programmieren\Python\DocDigTransformer\lib\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-3.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-4.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-5.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-6.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-7.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-8.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-9.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-10.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\0-11.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPR

Error processing file 9-7.png: string index out of range
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-8.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-9.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-10.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-11.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-12.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-13.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-14.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-15.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\9-16.png
C:\Uni\1M. Semeste

Error processing file 21-32.png: string index out of range
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-33.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-34.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-35.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-36.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-37.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-38.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-39.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-40.png
C:\Uni\1M. Semester\DocDig\gen_img\scan_1972_CdB_3_20231125160810.pdf\cells\NOPRE-COLOR-col-1\21-41.png
C:\Un

In [63]:
correct = []
wrong = []
# Display each image with its recognized text and best fuzzy match
#fig, axes = plt.subplots(nrows=len(ocr_outputs), ncols=1, figsize=(10, 5 * len(ocr_outputs)))
#fig.suptitle("OCR Outputs and Fuzzy Matches", fontsize=16)

for idx, item in enumerate(ocr_outputs):
    #ax = axes[idx] if len(ocr_outputs) > 1 else axes  # Handles single image case
    #ax.imshow(item["image"])
    #ax.axis('off')

    # Display fuzzy match and weighted fuzzy match below each image
    recognized_text = item["recognized_text"]
    true_text = item["true_text"]
    detected_color = item["detected_color"]
    detected_quoMark = item["detected_quoMark"]
    best_match = item["best_match"]
    score = item["score"]
    filtered_text = item["filtered_text"]
    best_match_filtered = item["best_match_filtered"]
    score_filtered = item["score_filtered"]
    best_match_customFuzzy = item["best_match_customFuzzy"]
    score_custom_fuzzy = item["score_customFuzzy"]
    
    if true_text == best_match_filtered:
        correct.append(score_filtered)
    else:
        wrong.append(score_filtered)
    
    text = f"True Text: '{true_text}'\n"
    text += f"Detected Color: '{detected_color}', Detected QuoMark: '{detected_quoMark}'\n"
    text += f"\nRecognized Text: '{recognized_text}'\n"
    text += f"Filtered Recognized Text: '{filtered_text}'\n"
    if best_match:
        text += f"Best Match for recognized text: '{best_match}' with Score: {score}\n"
    if best_match_filtered:
        text += f"Best Match for filtered text: '{best_match_filtered}' with Score: {score_filtered}\n"
    if best_match_customFuzzy:
        text += f"Best Match for filtered text with weight: '{best_match_customFuzzy}' with Score: {score_custom_fuzzy}\n"
    else:
        text += f"No confident match found (Score: {score})"

    #ax.set_title(text, fontsize=12)

#plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to fit title
#plt.show()

print("Corrects: ", len(correct), " Wrongs: ", len(wrong), " in Percentages: ", len(correct) / (len(correct) + len(wrong)))

Corrects:  742  Wrongs:  392  in Percentages:  0.654320987654321
